[View in Colaboratory](https://colab.research.google.com/github/mzhang32/nanohub-project/blob/master/first_submodel.ipynb)

#First Submodel
Refer to the design document [here](https://docs.google.com/document/d/1BM18EP8cWlJ371yYBoru_q8MHjesn3eBKg7CYRqPmv0/edit). We're using GitHub to back up and Google Drive to collaborate. I believe this tool(Google Colab) will be useful because it is highly collaborative and easy to use.

In [0]:
# Currently not being used.
!pip install PyOpenGL PyOpenGL_accelerate


In [0]:
 !pip install shapely



###Given a set of inputs about angle distribution and nanotube length and filament size, generate some line segments .


In [0]:
import numpy as np
import math
from shapely.geometry import LineString


def execute(volume_fraction, width_of_poly, length_of_poly, diameter, volume_of_poly, mean_nanowire_length, SD):
    
    # Find Volume of Nanowire
    def wire_volume(length, diameter):
        volume = math.pi * ((diameter / 2) ** 2) * length
        return volume 

    # Creating a list of random starting points for number of desired nanowires in sim
    def get_initial_point():
        x = np.random.uniform(-(width_of_poly / 2), (width_of_poly / 2))
        y = np.random.uniform(0, length_of_poly)
        z = np.random.uniform(0, width_of_poly)
        g = [x, y, z]
        nonlocal start_points
        start_points.append(g)
        
    # Creating lists of random lengths and angles     
    def lengths_angles():
        length = np.random.normal(mean_nanowire_length, SD)
        lengths.append(length)
        horiz_angle = np.random.uniform(0, math.pi)
        verti_angle = np.random.uniform(0, math.pi)
        a = [horiz_angle, verti_angle]
        angles.append(a)
          
    def check_distance(start_points, end_points, line1, diameter):
        for start, end in zip(start_points, end_points):
            point1 = (start[0], start[1], start[2])
            point2 = (end[0], end[1], end[2])
            line2 = LineString([point1, point2])
            dis = line1.distance(line2)
            if line1.distance(line2) < diameter:
              return False
        return True

    # Creating list of respective end points using initial points, lengths & angles      
    def get_end_points(start_points, lengths, angles, width_of_poly, length_of_poly, diameter):
        nonlocal wires
        x = start_points[wires][0]
        y = start_points[wires][1]
        z = start_points[wires][2]        
        phi = angles[wires][0]
        theta = angles[wires][1]
        length = lengths[wires]
        x1 = x + length * math.sin(theta) * math.sin(phi)
        y1 = y + length * math.cos(theta)
        z1 = z + length * math.sin(theta) * math.cos(phi)
        line1 = LineString([(x, y, z), (x1, y1, z1)])    
        print(wires)
        # Checks to make sure no internal creation
        if len(end_points) != 0:
            while check_distance(start_points, end_points, line1, diameter) == False:
                x = np.random.uniform(0, width_of_poly)
                y = np.random.uniform(0, width_of_poly)
                z = np.random.uniform(-(width_of_poly / 2), (width_of_poly / 2))
                g = [x, y, z]
                start_points[wires] = g
                x1 = x + length * math.sin(theta) * math.cos(phi)
                y1 = y + length * math.sin(theta) * math.sin(phi)
                z1 = z + length * math.cos(theta)      
                line1 = LineString([(x, y, z), (x1, y1, z1)])    
        # Checks to make sure end points are in domain
        while y1 > length_of_poly or z1 > width_of_poly or z1 < 0 or x1 > abs(width_of_poly / 2):
            x = np.random.uniform(0, width_of_poly)
            y = np.random.uniform(0, width_of_poly)
            z = np.random.uniform(-(width_of_poly / 2), (width_of_poly / 2))
            g = [x, y, z]
            start_points[wires] = g
            x1 = x + length * math.sin(theta) * math.cos(phi)
            y1 = y + length * math.sin(theta) * math.sin(phi)
            z1 = z + length * math.cos(theta)
        new = [x1, y1, z1]
        end_points.append(new)
        wires += 1    
        
    wires = 0
    volume = 0
    start_points = []
    lengths = []
    angles = []
    end_points = []
    
    while (volume / volume_of_poly) < volume_fraction:
        get_initial_point()
      
        lengths_angles()
        
        get_end_points(start_points, lengths, angles, width_of_poly, length_of_poly, diameter)
        
        volume += wire_volume(lengths[-1], diameter)
        print(volume)
       
    
    print(volume)
    return start_points, end_points
    
        
          
    


In [0]:
start, end = execute((2/100), .001, .001, 60e-9, 1e-9, 40e-6, 10e-6)

 ### Graphically visualize the model.
 We may have to import some libraries. A good-looking one is this python wrapper for OpenGL, [PyOpenGL](http://pyopengl.sourceforge.net/).
 
 It turns out using opengl in python notebook is not possible(?).

In [0]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

segments = []
for start, end in zip(start_points, end_points):
   x = [start[0], end[0]]
   y = [start[1], end[1]]
   z = [start[2], end[2]]
   segments.append([x, y, z])
for seg in segments:
  ax.plot(seg[0], seg[1], seg[2])
#ax.plot([0, -1], [0, -1], [0, -1]) # Test line segment
plt.show()  


###Represent the distribution of the nanowires and the distances between them as a graph.
Let G = (V, E) where V is the set of indiviual nanowires and let E = (V1, V2), the distance between nanowires V1 and V2. Using the graph package library [NetworkX](https://networkx.github.io/documentation/stable/tutorial.html).

In [0]:
!pip install networkx

In [0]:
import networkx as nx

""" Returns a networkx graph G formed from V = the segments in format 
 [(x1, x2), (y1, y2), (z1, z2)] and E = distances between the segments 
 that are within tolerance.
"""
def init_graph(segments, tol)
  number_of_tubes = len(segments)
  G = nx.Graph()
  # Sort by the one of the segment's endpoint's y value.
  V = sorted(segments, key=lambda v: v[1][0]) 
  line_strings = [LineString([(seg[0][0], seg[1][0], seg[2][0]), 
                         (seg[0][1], seg[1][1], seg[2][1])]) for seg in V]

  G.add_nodes_from(range(number_of_tubes))

  # Generate a graph using distances as weights. May 
  # need to optimize later.
  for i in range(number_of_tubes - 1):
    seg1 = V[i]
    line1 = line_strings[i]
    for j in range(i):
      seg2 = V[j]
      line2 = line_strings[j]
      d = line1.distance(line2)
      if d <= tolerance:
        G.add_edge(i, j, weight=line1.distance(line2))  
  return G
      
  

### Use the graph to compute a conductive path.
1. Graph theory is great. Details are in the working. The paper used the Union-Find Algorithm :
*"To find the conductive cluster in a nanotube system
between two electrodes and to calculate the probability of
percolation, an algorithm based on the graph theory is used.
According to the algorithm, all tubes of the system are
represented as vertices of a simple graph, edges of which
correspond to the connections between the tubes. Nanotubes
that are in contact with the opposite sides of the parallelepiped
form two subsets corresponding to the electrode. At the next
stage, a component that combines these subsets is sought by
means of the union-find algorithm. Thus, a conductive path in
the nanotubes system is formed.""* From [Computer Simulation of Field-Controlled Percolation
in 3D System of Straight Nanotubes](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8365699&tag=1).

2. Add in model for electrical conductivity of nanowires to calculate material properties in domain.


In [0]:
class UnionFind:
    """Union-find data structure.

    Each unionFind instance X maintains a family of disjoint sets of
    hashable objects, supporting the following two methods:

    - X[item] returns a name for the set containing the given item.
      Each set is named by an arbitrarily-chosen one of its members; as
      long as the set remains unchanged it will keep the same name. If
      the item is not yet part of a set in X, a new singleton set is
      created for it.

    - X.union(item1, item2, ...) merges the sets containing each item
      into a single larger set.  If any item is not yet part of a set
      in X, it is added to X as one of the members of the merged set.

      Union-find data structure. Based on Josiah Carlson's code,
      http://aspn.activestate.com/ASPN/Cookbook/Python/Recipe/215912
      with significant additional changes by D. Eppstein.
      http://www.ics.uci.edu/~eppstein/PADS/UnionFind.py

    """

    def __init__(self):
        """Create a new empty union-find structure."""
        self.weights = {}
        self.parents = {}

    def __getitem__(self, object):
        """Find and return the name of the set containing the object."""

        # check for previously unknown object
        if object not in self.parents:
            self.parents[object] = object
            self.weights[object] = 1
            return object

        # find path of objects leading to the root
        path = [object]
        root = self.parents[object]
        while root != path[-1]:
            path.append(root)
            root = self.parents[root]

        # compress the path and return
        for ancestor in path:
            self.parents[ancestor] = root
        return root
        
    def __iter__(self):
        """Iterate through all items ever found or unioned by this 
        structure."""
        return iter(self.parents)

    def union(self, *objects):
        """Find the sets containing the objects and merge them all."""
        roots = [self[x] for x in objects]
        heaviest = max([(self.weights[r],r) for r in roots])[1]
        for r in roots:
            if r != heaviest:
                self.weights[heaviest] += self.weights[r]
                self.parents[r] = heaviest

In [0]:
""" Given the length of the cross-section, a list of sorted segments 
 [(x1, x2), (y1, y2), (z1, z2)], graph G, and a tolerance, return a 
 connecting graph. If none, return None. 
"""
def find_connecting_path(length, segs, G, tol):

  source_plane = length / 20
  sink_plane = length - length / 20

  H = nx.Graph()
  #X = UnionFind()

  def is_between(b1, x, b2):
    return (b1 <= x and x >= b2) or (b2 <= x and x >= b1)
    

  n = range(len(segs))
  source_segs = [i for i in n if is_between(segs[i][1][0], 
                                            source_plane, seg[i][1][1])]
  sink_segs = [i for i in n if is_between(segs[i][1][0], 
                                          sink_plane, seg[i][1][1])]
  
  if not source_segs or not sink_segs:
    print("No nanotubes intersecting source and/or sink planes.")
    return None
  
  H.add_nodes_from(source_segs)
  #X.union(source_segs)

  new_segs = source_segs
  while new_segs:
    # Add contacting tubes to each set.  
    temp = list(new_segs)
    new_segs = []
    for seg in temp:
      for v in G.neighbors(seg):
        if v not in source_segs:
          #X.union(source_segs[0], v)
          new_segs.append(v)
          H.add_edge(seg, temp)
          if v in sink_segs:
            print("Connecting path found, here is the cluster.")
            return H
        

  print("No connecting path found.")
  return None

    



###Graphically visualize the model with the path.

In [0]:
plt.subplot(121)
nx.draw(G)
